<a href="https://www.kaggle.com/code/caasperart/learnfft-spectogram?scriptVersionId=123450455" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib.cm as cm

from scipy import signal
from scipy.fft import fft, fftfreq

figure(figsize=(10,10))


In [ ]:
cc = pd.read_csv('/kaggle/input/ccs170/ccs170.csv')
# cc = pd.read_csv('/kaggle/input/ccsv122/ccs122.csv')
# cc = pd.read_csv('/kaggle/input/ccs001/mcsadc-motor-rotorbarfailure-1_2023/ccs10.csv')

cc.columns = ['time', 'ia', 'ib', 'ic', 'va', 'vb', 'vc', 'rad/s', 'pos/mec' ]
ia = cc['ia']

ia = np.array(ia)

fs = 1428
f,t,sxx = signal.stft(ia, fs=fs)
plt.subplot(121)
plt.pcolormesh(t, f[:56], np.abs(sxx[:56, :]), cmap='gray')
plt.xlabel("time s")
plt.ylabel("Frequency (Hz)")

plt.subplot(122)
plt.pcolormesh(t, f, np.abs(sxx), cmap='gray')
plt.xlabel("time s")
plt.ylabel("Frequency (Hz)")

In [ ]:

ib = np.array(cc['ib'])
yb = fft(ib, norm="forward")
xb = fftfreq(len(ib), 1.0/1428.0)

plt.subplot(211)
plt.plot(xb[0:len(ib)//2], np.abs(yb[0:len(ib)//2]))


perfft = ((len(xb)) / 1428.0)
plt.subplot(212)
plt.xlim(40, 60)
plt.ylim(0, 0.2)
plt.plot(xb[0:len(ib)//2], np.abs(yb[0:len(ib)//2]))
# plt.plot(range(round(perfft*60),len(xb)//2), np.abs(yb[round(perfft*60):len(yb)//2]))


In [ ]:
fs = 1428.0
f0 = 50.0
q  = 25

bNumerator, aDenominator = signal.iirnotch(f0, q, fs)

ibFiltered = signal.filtfilt(bNumerator, aDenominator, ib)

cc1 = pd.read_csv('/kaggle/input/ccs001/mcsadc-motor-rotorbarfailure-1_2023/ccs10.csv')

cc1.columns = ['time', 'ia', 'ib', 'ic', 'va', 'vb', 'vc', 'rad/s', 'pos/mec' ]
ia = cc['ia']
iHealtyFiltered = signal.filtfilt(bNumerator, aDenominator, ia)
yhealty = fft(iHealtyFiltered, norm='forward')
xhealty = fftfreq(len(iHealtyFiltered), 1.0/fs)#[0:len(xbf)//2]

ybf = fft(ibFiltered, norm='forward')
xbf = fftfreq(len(ibFiltered), 1.0/fs)#[0:len(xbf)//2]

plt.rcParams["figure.figsize"] = (10,20)

plt.subplot(311)
plt.plot(xbf[0:len(xbf)//2], np.abs(ybf[0:len(ybf)//2]))

f,t,sxx = signal.stft(ibFiltered, fs=fs)
plt.subplot(312)
plt.pcolormesh(t, f, np.abs(sxx), cmap='gray')
plt.xlabel("time s")
plt.ylabel("Frequency (Hz)")

plt.subplot(313)
f, h = signal.freqz(bNumerator, aDenominator, fs=fs)
plt.plot(f, 20*np.log10(abs(h)))



In [ ]:
deviation = yhealty-yb

plt.rcParams["figure.figsize"] = (30,10)

f,t,sxx = signal.stft(ibFiltered, fs=fs)
f1,t1,sxx1 = signal.stft(iHealtyFiltered, fs=fs)

diference = np.abs(sxx) - np.abs(sxx1)

plt.subplot(131)
plt.title("2BRB")
plt.pcolormesh(t,f, abs(sxx), cmap='gray')

plt.subplot(132)
plt.title("Healty")
plt.pcolormesh(t1,f1, abs(sxx1), cmap='gray')

plt.subplot(133)
plt.title("Diference")
plt.pcolormesh(t,f, np.abs(diference), cmap='gray')


In [ ]:
plt.rcParams['figure.figsize'] = (10,5)
plt.specgram(ia, Fs=fs, scale='dB', cmap='gray')

# Gradio Learning Section 


In [3]:
! pip install gradio -q

In [4]:
# %reload_ext gradio
%load_ext gradio

In [6]:
%%blocks

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np

import gradio as gr
def markPort(port):
    if port == '':
        return
    
    return f"port: {port}"

def plot_forecast(final_year, companies, noise, show_legend, point_style):
    start_year = 2020
    x = np.arange(start_year, final_year + 1)
    year_count = x.shape[0]
    plt_format = ({"cross": "X", "line": "-", "circle": "o--"})[point_style]
    fig = plt.figure()
    ax = fig.add_subplot(111)
    for i, company in enumerate(companies):
        series = np.arange(0, year_count, dtype=float)
        series = series**2 * (i + 1)
        series += np.random.rand(year_count) * noise
        ax.plot(x, series, plt_format)
    if show_legend:
        plt.legend(companies)
    return fig


gr.Markdown("# MoMos - Motor Monitoring System Dashboard")
bl = gr.Blocks()
with bl:
    
    with gr.Tab("Setting"):
        with gr.Column():
            gr.Markdown("### Port: ")
            portRadio = gr.Radio(['com5', 'com6', 'com7', 'com8', 'com9'], label="Choose")
            port = gr.Textbox(show_label=False, max_lines=1, placeholder="Choose or insert")
            displayPort = gr.Markdown()
            
            portRadio.change(markPort, portRadio, displayPort)
            port.submit(markPort, port, displayPort)
            
    with gr.Tab("Display"):
        with gr.Column():
            gr.Markdown("### display plot Here!!")
            year = gr.Radio([2025, 2030, 2035, 2040], label="Project to:")
            group = gr.CheckboxGroup(["Google", "Microsoft", "Gradio"], label="Company Selection")
            noise = gr.Slider(1, 100, label="Noise Level")
            showLegend = gr.Checkbox(label="Show Legend")
            style = gr.Dropdown(["cross", "line", "circle"], label="Style")
            
            showPlot = gr.Button("Plot")
            
            plotPlace = gr.Plot(label="forecast")
            
            showPlot.click(plot_forecast,inputs=[year,group,noise,showLegend,style],outputs=plotPlace)
# bl.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://31f636c89c9df0dd73.gradio.live


Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/gradio/routes.py", line 401, in run_predict
    event_data=event_data,
  File "/opt/conda/lib/python3.7/site-packages/gradio/blocks.py", line 1076, in process_api
    fn_index, inputs, iterator, request, event_id, event_data
  File "/opt/conda/lib/python3.7/site-packages/gradio/blocks.py", line 885, in call_function
    fn, *processed_input, limiter=self.limiter
  File "/opt/conda/lib/python3.7/site-packages/anyio/to_thread.py", line 32, in run_sync
    func, *args, cancellable=cancellable, limiter=limiter
  File "/opt/conda/lib/python3.7/site-packages/anyio/_backends/_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "/opt/conda/lib/python3.7/site-packages/anyio/_backends/_asyncio.py", line 867, in run
    result = context.run(func, *args)
  File "<string>", line 16, in plot_forecast
NameError: name 'np' is not defined
Traceback (most recent call last):
  File "/opt/conda

## issues: https://github.com/gradio-app/gradio/issues/3625

sepertinya gradio tidak bisa mendeteksi library yang sudah diimport hanya ketika menggunakan `%%blocks` saja, ketika langsung menggunakan `Block().Launch` gradio bekerja dengan semestinya.
